Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')


In [0]:
import numpy as np

# Wrangle the modeling data

# indicate_missing is a function that returns a boolean value if the inbound data = 'MISSING'
#   - helps in creating a "missing" column 
def indicate_missing(val):
  if val == 'MISSING':
    return True

  return False

# boolean_missing is a function converting the permit boolean column to categorical data 
def boolean_missing(val):
  if val == True:
    return 'TRUE'

  if val == False:
    return 'FALSE'

  return 'MISSING'

def wrangle(DF):
  X = DF.copy()

  # Replace near zero latitude values with zero
  X['latitude'] = X['latitude'].replace(-2e-08, 0)

  # Replace zero values with nan so we can impute values downstream
  cols_with_zeroes = ['longitude',
                      'latitude',
                      'construction_year',
                      'gps_height',
                      'population']
  for col in cols_with_zeroes:
    X[col] = X[col].replace(0, np.nan)   # replace zeros with nans
    X[col+'_MISSING'] = X[col].isnull()  # create a missing indicator column

  # Create columns for month and year recorded data
  X['date_recorded']  = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
  X['year_recorded']  = X['date_recorded'].dt.year
  X['month_recorded'] = X['date_recorded'].dt.month

  # Create a column reflecting the number of years from construction to year recorded
  X['years']          = X['year_recorded'] - X['construction_year']
  X['years']          = X['years'].replace(0, np.nan)   # replace zeros with nans
  X['years_MISSING']  = X['years'].isnull() # create a missing years indicator column

  # Replace missing boolean data with categorical data reflecting that missing data
  cols_boolean_missing = ['public_meeting', 'permit']
  for col in cols_boolean_missing:
    X[col+'_CATEGORICAL']             = X[col].apply(boolean_missing)
    X[col+'_CATEGORICAL'+'_MISSING']  = X[col+'_CATEGORICAL'].apply(indicate_missing)

  # Replace missing categorical data with 'MISSING'
  cols_categorical_missing = ['funder', 'installer', 'scheme_name', 'scheme_management', 'subvillage']
  for col in cols_categorical_missing:
    X[col]            = X[col].replace(np.nan, 'MISSING')   # replace zeros with nans
    X[col+'_MISSING'] = X[col].apply(indicate_missing)

  # List columns to be dropped
  cols_drop = [
               'date_recorded',             # date_recorded - using year_recorded and month_recorded instead
               'quantity_group',            # duplicate column
               'payment_type',              # duplicate column
               'recorded_by',               # data collection process column (not predictive)
               'id',                        # data collection process column (not predictive)
               'permit',                    # replaced by categorical column: permit_string
               'num_private',               # 98% zeroes, unclear the purpose of this dat
               'construction_year',         # use 'years' as a proxy
               'construction_year_MISSING', # use 'years_MISSING' as a proxy
               'amount_tsh']                # highly skewed data
  # Also drop the columns we processed due to missing values
  cols_drop.extend(cols_boolean_missing)

  # Drop undesired columns
  X = X.drop(columns=cols_drop)

  return X

In [64]:
# Wrange the initial train and test datasets
df_train = wrangle(train)
df_test  = wrangle(test)

print(f'Train shape: {df_train.shape} Test shape: {df_test.shape}')

Train shape: (59400, 48) Test shape: (14358, 47)


In [0]:
# Import pandas profiling package
from pandas_profiling import ProfileReport

# Generate and download profile reports for the wrangled train and test datasets
ProfileReport(df_train, minimal=True).to_file(output_file="tanzania_train_output.html")
ProfileReport(df_test,  minimal=True).to_file(output_file="tanzania_test_output.html")

In [0]:
# The status_group column is the target
target = 'status_group'

# Define the training model inputs and target columns
features = df_train.columns.drop(target)
X_train  = df_train[features]
y_train  = df_train[target]

In [77]:
y_train.sample(10)

58140    functional needs repair
1903                  functional
14770             non functional
39142             non functional
27959                 functional
19097                 functional
45028             non functional
44595                 functional
18192                 functional
58882                 functional
Name: status_group, dtype: object

In [0]:
import category_encoders as ce 
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

# Construct the Cross Validation pipeline and fold configuration
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='mean'),
    RandomForestClassifier(random_state=42)
)


In [0]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

# Configure ranges of hyperparameter distributions
param_dist = {
    'randomforestclassifier__n_estimators': randint(50, 500),
    'randomforestclassifier__max_depth': [5, 10, 15, 20, None],
    'randomforestclassifier__max_features': uniform(0, 1)
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_dist,
    n_iter=10,
    cv=3,
    scoring=None,
    verbose=10,
    return_train_score=True,
    n_jobs=-1
)

In [93]:
y_train.sample(5)

51370        functional
1558         functional
55650        functional
46015    non functional
51790        functional
Name: status_group, dtype: object

In [94]:
# Fit the model against the selected parameters
search.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   28.4s
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  7.8min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed: 19.4min
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 28.1min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('ordinalencoder',
                                              OrdinalEncoder(cols=None,
                                                             drop_invariant=False,
                                                             handle_missing='value',
                                                             handle_unknown='value',
                                                             mapping=None,
                                                             return_df=True,
                                                             verbose=0)),
                                             ('simpleimputer',
                                              SimpleImputer(add_indicator=False,
                                                            copy=True,
                                                            fill_value=None,


In [95]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation MAE', -search.best_score_)

Best hyperparameters {'randomforestclassifier__max_depth': 15, 'randomforestclassifier__max_features': 0.3643287595003505, 'randomforestclassifier__n_estimators': 336}
Cross-validation MAE -0.7962289562289562


In [0]:
# Generate predictions
pipeline_best = search.best_estimator_

X_test = df_test[features]

# Generate model 'status_group' predictions for test dataset
y_pred = pipeline_best.predict(X_test)

In [0]:
# Construct dataframe housing the Kaggle submission dataset
tmp_dict = {'id': list(test['id']), 'status_group': list(y_pred)}
df_submission = pd.DataFrame(tmp_dict)

In [0]:
# Create submission csv (download and submit to Kaggle)
df_submission.to_csv("submission_dfa.csv", index=False)